In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from autogluon.tabular import TabularPredictor
import matplotlib.pyplot as plt
import os

In [3]:
# remove models folder if it exists
import shutil
shutil.rmtree("../models", ignore_errors=True)

In [4]:
def prepare_data(true_data, counter_files, max_index):
    combined_data = {}
    # Define the column names to keep.
    cols = [f"B{i}" for i in range(1, max_index + 1)]

    # Process the true data: retain only the B columns and add the true label.
    true_data['label'] = "true"
    true_data = true_data[cols + ["label"]]

    for key, df_counter in counter_files.items():
        # Retain only the B columns and add the false label.
        df_counter['label'] = "false"
        df_counter = df_counter[cols + ["label"]]
        # Combine the true data with this counter example.
        combined = pd.concat([true_data, df_counter], ignore_index=True)
        combined_data[key] = combined

    return combined_data

In [5]:
presets = ['good_quality', 'optimize_for_deployment']

def generateModels(combined_data, prefix, max_index, time_limit=180):
    results = {}
    # Define the feature columns
    feature_cols = [f"B{i}" for i in range(1, max_index + 1)]

    for key, df in combined_data.items():
        # Split data into features and label.
        X = df[feature_cols]
        y = df["label"]

        # Train-test split: 70% train, 30% test.
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
        train_data = pd.concat([X_train, y_train], axis=1)
        test_data = pd.concat([X_test, y_test], axis=1)

        # Define an output folder for saving models for this dataset variant.
        output_folder = f"../models/{prefix}/{key}"
        os.makedirs(output_folder, exist_ok=True)

        # Train the model using AutoGluon.
        predictor = TabularPredictor(label="label", path=output_folder, eval_metric='f1').fit(train_data, presets=presets, time_limit=time_limit)

        # Evaluate the model on the test data.
        leaderboard = predictor.leaderboard(test_data, silent=True)
        results[key] = leaderboard

        print(f"Results for {key}:")
        print(leaderboard)

    # Create results folder and save leaderboards.
    results_folder = f"../models/{prefix}/results"
    os.makedirs(results_folder, exist_ok=True)
    for key, result in results.items():
        result.to_csv(os.path.join(results_folder, f"{key}_leaderboard.csv"), index=False)

In [6]:
true_file = "../data/ei/data_ei.csv"
ei_true_data = pd.read_csv(true_file)

# List of counter example files for the EI zone
counter_files = {
    "combined": pd.concat(
        [
            pd.read_csv("../data/ei/data_ei_random.csv"),
            pd.read_csv("../data/ei/data_ez_counter_example.csv"),
            pd.read_csv("../data/ei/data_ie_counter_example.csv"),
            pd.read_csv("../data/ei/data_ze_counter_example.csv"),
            pd.read_csv("../data/ei/data_ie_true_counter_example.csv"),
            pd.read_csv("../data/ei/data_test_false.csv"),
         ]
    )
}

# For EI zone, our CSV files have columns B1 to B12.
combined_data = prepare_data(ei_true_data, counter_files, max_index=12)

# Dictionary to store the combined DataFrames and later model results
generateModels(combined_data, 'ei', 12, time_limit=21600)

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.9.13
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26100
CPU Count:          16
Memory Avail:       6.45 GB / 15.35 GB (42.0%)
Disk Space Avail:   47.75 GB / 100.00 GB (47.7%)
Presets specified: ['good_quality', 'optimize_for_deployment']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Note: `save_bag_folds=False`! This will greatly reduce peak disk usage during fit (by ~8x), but runs the risk of an out-of-memory error during model refit if memory is small relative to the data size.
	You can avoid this risk by setting `save_bag_folds=True`.
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected 

(_ray_fit pid=22400) [1000]	valid_set's binary_logloss: 0.063705	valid_set's f1: 0.957524
(_ray_fit pid=22400) [2000]	valid_set's binary_logloss: 0.0596066	valid_set's f1: 0.962963 [repeated 8x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)
(_ray_fit pid=22400) [3000]	valid_set's binary_logloss: 0.058624	valid_set's f1: 0.963412 [repeated 8x across cluster]
(_ray_fit pid=22400) [4000]	valid_set's binary_logloss: 0.0589464	valid_set's f1: 0.965462 [repeated 8x across cluster]
(_ray_fit pid=22400) [5000]	valid_set's binary_logloss: 0.0598116	valid_set's f1: 0.965489 [repeated 7x across cluster]
(_ray_fit pid=22400) [6000]	valid_set's binary_logloss: 0.0608523	valid_set's f1: 0.966505 [repeated 7x across cluster]
(_ray_fit pid=22400) [7000]	valid_set's binary_logloss: 0.0619875	valid_set's f1: 0.966714 [repeat

(_dystack pid=14208) 	0.9675	 = Validation score   (f1)
(_dystack pid=14208) 	132.8s	 = Training   runtime
(_dystack pid=14208) 	51.79s	 = Validation runtime
(_dystack pid=14208) Fitting model: LightGBM_BAG_L1 ... Training model for up to 3442.75s of the 5240.25s of remaining time.
(_dystack pid=14208) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=2, gpus=0, memory=0.23%)


(_ray_fit pid=8328) [1000]	valid_set's binary_logloss: 0.0611877	valid_set's f1: 0.960582
(_ray_fit pid=24548) [1000]	valid_set's binary_logloss: 0.0657267	valid_set's f1: 0.955569
(_ray_fit pid=8328) [2000]	valid_set's binary_logloss: 0.0575243	valid_set's f1: 0.965253 [repeated 7x across cluster]
(_ray_fit pid=8328) [3000]	valid_set's binary_logloss: 0.0570503	valid_set's f1: 0.96631 [repeated 7x across cluster]
(_ray_fit pid=8328) [4000]	valid_set's binary_logloss: 0.0580491	valid_set's f1: 0.96713 [repeated 7x across cluster]
(_ray_fit pid=8328) [5000]	valid_set's binary_logloss: 0.059545	valid_set's f1: 0.966895 [repeated 7x across cluster]
(_ray_fit pid=20476) [6000]	valid_set's binary_logloss: 0.0584109	valid_set's f1: 0.966465 [repeated 7x across cluster]
(_ray_fit pid=24256) [7000]	valid_set's binary_logloss: 0.061186	valid_set's f1: 0.969855 [repeated 6x across cluster]
(_ray_fit pid=24256) [8000]	valid_set's binary_logloss: 0.0626911	valid_set's f1: 0.970636 [repeated 4x acr

(_dystack pid=14208) 	0.9675	 = Validation score   (f1)
(_dystack pid=14208) 	135.31s	 = Training   runtime
(_dystack pid=14208) 	45.56s	 = Validation runtime
(_dystack pid=14208) Fitting model: RandomForestGini_BAG_L1 ... Training model for up to 3294.71s of the 5092.22s of remaining time.
(_dystack pid=14208) 	0.9627	 = Validation score   (f1)
(_dystack pid=14208) 	2.53s	 = Training   runtime
(_dystack pid=14208) 	1.67s	 = Validation runtime
(_dystack pid=14208) Fitting model: RandomForestEntr_BAG_L1 ... Training model for up to 3290.31s of the 5087.81s of remaining time.
(_dystack pid=14208) 	0.9614	 = Validation score   (f1)
(_dystack pid=14208) 	2.03s	 = Training   runtime
(_dystack pid=14208) 	1.68s	 = Validation runtime
(_dystack pid=14208) Fitting model: CatBoost_BAG_L1 ... Training model for up to 3286.40s of the 5083.90s of remaining time.
(_dystack pid=14208) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=2, gpus=0

(_ray_fit pid=16784) [1000]	valid_set's binary_logloss: 0.0566452	valid_set's f1: 0.962888
(_ray_fit pid=16784) [2000]	valid_set's binary_logloss: 0.0558062	valid_set's f1: 0.965337 [repeated 8x across cluster]
(_ray_fit pid=16784) [3000]	valid_set's binary_logloss: 0.0568761	valid_set's f1: 0.967104 [repeated 7x across cluster]


(_ray_fit pid=16784) 	Ran out of time, early stopping on iteration 3371. Best iteration is:
(_ray_fit pid=16784) 	[3366]	valid_set's binary_logloss: 0.0576004	valid_set's f1: 0.968094
(_ray_fit pid=23988) 	Ran out of time, stopping training early. (Stopping on epoch 78) [repeated 7x across cluster]
(_dystack pid=14208) 	0.9685	 = Validation score   (f1)
(_dystack pid=14208) 	61.51s	 = Training   runtime
(_dystack pid=14208) 	107.11s	 = Validation runtime
(_ray_fit pid=13232) 	Ran out of time, early stopping on iteration 3303. Best iteration is: [repeated 6x across cluster]
(_ray_fit pid=13232) 	[2885]	valid_set's binary_logloss: 0.0579046	valid_set's f1: 0.965852 [repeated 6x across cluster]
(_dystack pid=14208) Fitting model: WeightedEnsemble_L2 ... Training model for up to 360.00s of the 1790.53s of remaining time.
(_dystack pid=14208) 	Ensemble Weights: {'NeuralNetTorch_BAG_L1': 0.3, 'LightGBMXT_BAG_L1': 0.15, 'LightGBM_BAG_L1': 0.15, 'RandomForestGini_BAG_L1': 0.15, 'RandomForestEn

Results for combined:
                           model  score_test score_val eval_metric  \
0   RandomForestEntr_BAG_L2_FULL    0.973865      None          f1   
1      LightGBMLarge_BAG_L1_FULL    0.972153      None          f1   
2            XGBoost_BAG_L1_FULL    0.971640      None          f1   
3         LightGBMXT_BAG_L1_FULL    0.970114      None          f1   
4           LightGBM_BAG_L1_FULL    0.969751      None          f1   
5     NeuralNetTorch_BAG_L1_FULL    0.968885      None          f1   
6   RandomForestGini_BAG_L1_FULL    0.964867      None          f1   
7           CatBoost_BAG_L1_FULL    0.964247      None          f1   
8   RandomForestEntr_BAG_L1_FULL    0.962901      None          f1   
9     ExtraTreesGini_BAG_L1_FULL    0.956632      None          f1   
10    ExtraTreesEntr_BAG_L1_FULL    0.954728      None          f1   
11   NeuralNetFastAI_BAG_L1_FULL    0.915713      None          f1   

    pred_time_test  pred_time_val     fit_time  pred_time_test_marg

In [ ]:
true_file = "../data/ie/data_ie.csv"
ie_true_data = pd.read_csv(true_file)

# List of counter example files for the EI zone
counter_files = {
    "combined": pd.concat(
        [
            pd.read_csv("../data/ie/data_ie_random.csv"),
            pd.read_csv("../data/ie/data_ez_counter_example.csv"),
            pd.read_csv("../data/ie/data_ei_true_counter_example.csv"),
            pd.read_csv("../data/ie/data_ei_counter_example.csv"),
            pd.read_csv("../data/ie/data_ze_counter_example.csv"),
            pd.read_csv("../data/ie/data_test_false.csv"),
         ]
    )
}

# For EI zone, our CSV files have columns B1 to B105.
combined_data = prepare_data(ie_true_data, counter_files, max_index=105)

# Dictionary to store the combined DataFrames and later model results
generateModels(combined_data, 'ie', 105, time_limit=21600)

In [ ]:
rue_file = "../data/ez/data_ez.csv"
ez_true_data = pd.read_csv(true_file)

# List of counter example files for the EI zone
counter_files = {
    "combined": pd.concat(
        [
            pd.read_csv("../data/ez/data_ez_random.csv"),
            pd.read_csv("../data/ez/data_ei_counter_example.csv"),
            pd.read_csv("../data/ez/data_ie_counter_example.csv"),
            pd.read_csv("../data/ez/data_ze_counter_example.csv"),
            pd.read_csv("../data/ez/data_sample_combined.csv"),
         ]
    )
}

# For EI zone, our CSV files have columns B1 to B550.
combined_data = prepare_data(ez_true_data, counter_files, max_index=550)

# Dictionary to store the combined DataFrames and later model results
generateModels(combined_data, 'ez', 550, time_limit=21600)

In [ ]:
true_file = "../data/ze/data_ze.csv"
ze_true_data = pd.read_csv(true_file)

# List of counter example files for the EI zone
counter_files = {
    "combined": pd.concat(
        [
            pd.read_csv("../data/ze/data_ze_random.csv"),
            pd.read_csv("../data/ze/data_ez_counter_example.csv"),
            pd.read_csv("../data/ze/data_ie_counter_example.csv"),
            pd.read_csv("../data/ze/data_ei_counter_example.csv"),
            pd.read_csv("../data/ze/data_sample_combined.csv"),
         ]
    )
}

# For EZ zone, our CSV files have columns B1 to B550.
combined_data = prepare_data(ze_true_data, counter_files, max_index=550)

# Dictionary to store the combined DataFrames and later model results
generateModels(combined_data, 'ze', 550, time_limit=21600)